In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pyodbc
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import numpy as np

In [2]:
def database_connection():
    conn = pyodbc.connect(
        r'DRIVER={SQL Server Native Client 11.0};'
        r'SERVER=localhost;'
        r'DATABASE=seis603_pricing_project;'
        r'UID=python_dev;'
        r'PWD=python_dev;'
        )
    cursor = conn.cursor()
    return cursor

db_connection = database_connection()

In [3]:
#items tracking
sql = "Select * from dbo.Baseball_Stats"
db_connection.execute(sql)#, params) #executing sproc
list_items = db_connection.fetchall()#[0] #fetchone will only return first result
#df = pd.Series(list_items)
labels = ['pk','regular_season','post_season','year','league']
df = pd.DataFrame.from_records(list_items, columns=labels) #create dataframe from list

In [4]:
#converting to floats
df['regular_season'] = df['regular_season'].astype('float')
df['post_season'] = df['post_season'].astype('float')

In [35]:
#select * from airport_freq where airport_ident = 'KLAX' order by type desc
#airport_freq[airport_freq.airport_ident == 'KLAX'].sort_values('type', ascending=False)


data = df.sort_values('year', ascending=True)
#print(data.head())

al = df[df.league=='AL'][['regular_season','post_season','year']]
nl = df[df.league=='NL'][['regular_season','post_season','year']]

#AL avg
al_avg_reg = al.groupby(['year'])['regular_season'].mean()
al_avg_post = al.groupby(['year'])['post_season'].mean()

#NL Avg
nl_avg_reg = nl.groupby(['year'])['regular_season'].mean()
nl_avg_post = nl.groupby(['year'])['post_season'].mean()

al_join = pd.concat([al_avg_reg, al_avg_post], axis=1)
nl_join = pd.concat([nl_avg_reg, nl_avg_post], axis=1)







In [36]:
al_trace = go.Scatter(
    x = al_join['regular_season'],#dates
    y = al_join['post_season'],
    name = 'AL Ratio',
    mode = 'markers',
    line = dict(
        color = ('rgb(49, 86, 219)'),
        width = 4)
)

nl_trace = go.Scatter(
    x = nl_join['regular_season'],#dates
    y = nl_join['post_season'],
    name = 'NL Ratio',
    mode = 'markers',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)

In [37]:
#all the data i.e. the lines in the graph
data = [al_trace, nl_trace]

#Creating layout of report
layout = go.Layout(
    title='Compare Regular Season and Post Season Over Time',
    xaxis=dict(
        title='Regular Season',
        zeroline = False,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Post Season',
        zeroline = False,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    
)



fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-line')